# Part 11

## Assistant, Thread, Message, and Run Review
Let's make some objects, again!

Univeral code for the entire notebook

In [18]:
# Import necessary libraries
from openai import OpenAI  # Used for interacting with OpenAI's API
from typing_extensions import override  # Used for overriding methods in subclasses
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants

# Additional libraries for time and date manipulation
import time
import pytz
import datetime


In [19]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [20]:

# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    instructions="You are a helpful assistant.",  # Set the instructions for the assistant.
    name="Run Step Friendly Assistant",  # Give the assistant a name.
    metadata={  # Add metadata about the assistant's capabilities.
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
        "run_step_friendly": "True",
    },
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Print the details of the created assistant to check its properties.
print(assistant)  # Print the full assistant object.
print("\n\n")
print(assistant.name)  # Print the name of the assistant.
print(assistant.metadata)  # Print the metadata of the assistant.


Assistant(id='asst_yTqvGZiGrntdsm8J6WDC7xa8', created_at=1715941552, description=None, instructions='You are a helpful assistant.', metadata={'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True', 'run_step_friendly': 'True'}, model='gpt-4o', name='Run Step Friendly Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Run Step Friendly Assistant
{'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True', 'run_step_friendly': 'True'}


### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [21]:
# Create a thread using the OpenAI API and store it in a variable.
# The metadata specifies a user identifier.
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console.
print(thread)

Thread(id='thread_8lV62hFNMb0jlWQS4P2ovDir', created_at=1715941552, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [22]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Tell me what a penguin is in 100 words or less.",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

Message(id='msg_7rkgOVJm5vvUtEm7eJKN9P6F', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')], created_at=1715941553, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_8lV62hFNMb0jlWQS4P2ovDir')


msg_7rkgOVJm5vvUtEm7eJKN9P6F
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]
Tell me what a penguin is in 100 words or less.
user


### Creating Streaming Run
Let's create a run and get some output!

In [23]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize the results list

    @override
    def on_text_created(self, text) -> None:
        """Handle the event when text is first created."""
        print("\nassistant text > ", end="", flush=True)
        self.results.append(text)

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        print(delta.value, end="", flush=True)
        self.results.append(delta.value)

    def on_tool_call_created(self, tool_call):
        """Handle the event when a tool call is created."""
        print(f"\nassistant tool > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        """Handle the event when there is a delta (update) in a tool call."""
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
                self.results.append(delta.code_interpreter.input)
            if delta.code_interpreter.outputs:
                print("\n\noutput >", flush=True)
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    print(f"\n{output.logs}", flush=True)
                    self.results.append(output.logs)


In [24]:
event_handler = EventHandler()

with client.beta.threads.runs.stream(
    thread_id=thread.id,         # ID of the thread to run.
    assistant_id=assistant.id,   # ID of the assistant to use.
    event_handler=event_handler,  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()

# Retrieve the results from the event handler
results = event_handler.results

# Process or print the results as needed
print("\n\n")
print("Collected results:", results)




assistant text > A penguin is a flightless bird primarily found in the Southern Hemisphere, particularly in Antarctica. Known for their distinctive black and white plumage, penguins are excellent swimmers, using their flipper-like wings to navigate underwater. They feed mainly on fish, squid, and krill. Penguins are social animals, often forming large colonies for breeding and protection. Their unique adaptations, such as a layer of blubber and tightly packed feathers, provide insulation against the cold. There are around 18 species of penguins, varying in size and habitat preferences.


Collected results: [Text(annotations=[], value='A'), 'A', ' peng', 'uin', ' is', ' a', ' flight', 'less', ' bird', ' primarily', ' found', ' in', ' the', ' Southern', ' Hemisphere', ',', ' particularly', ' in', ' Antarctica', '.', ' Known', ' for', ' their', ' distinctive', ' black', ' and', ' white', ' plum', 'age', ',', ' peng', 'uins', ' are', ' excellent', ' swimmers', ',', ' using', ' their', ' f

## Listing Run Steps

thread_id
(string)

Required
The ID of the thread the run and run steps belong to.

run_id
(string)

Required
The ID of the run the run steps belong to.

limit
(integer)

Optional
Defaults to 20
A limit on the number of objects to be returned. Limit can range between 1 and 100, and the default is 20.

order
(string)

Optional
Defaults to desc
Sort order by the created_at timestamp of the objects. asc for ascending order and desc for descending order.

after
(string)

Optional
A cursor for use in pagination. after is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, ending with obj_foo, your subsequent call can include after=obj_foo in order to fetch the next page of the list.

before
(string)

Optional
A cursor for use in pagination. before is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, ending with obj_foo, your subsequent call can include before=obj_foo in order to fetch the previous page of the list.

In [25]:

# List all run steps for the specified thread and run
all_run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,          # ID of the thread
    run_id=stream.get_final_run().id  # ID of the final run in the stream
)

# Print all run steps to check their properties
print(all_run_steps)
print("\n")

# Iterate through each run step and print its details
for run_step in all_run_steps:
    print(run_step.id)           # Print the run step ID
    print(run_step.status)       # Print the run step status
    print(run_step.usage)        # Print the run step usage
    print(run_step.step_details) # Print the run step details
    print("\n")




SyncCursorPage[RunStep](data=[RunStep(id='step_MM92lE2fUssdWyXIpCKOcpbF', assistant_id='asst_yTqvGZiGrntdsm8J6WDC7xa8', cancelled_at=None, completed_at=1715941562, created_at=1715941559, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_g2Nf7E6dgUQslnsYf7DS8xjQ', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_CoWSwKPgXJhB5dyTMiTn8pG6'), type='message_creation'), thread_id='thread_8lV62hFNMb0jlWQS4P2ovDir', type='message_creation', usage=Usage(completion_tokens=112, prompt_tokens=44, total_tokens=156), expires_at=None)], object='list', first_id='step_MM92lE2fUssdWyXIpCKOcpbF', last_id='step_MM92lE2fUssdWyXIpCKOcpbF', has_more=False)


step_MM92lE2fUssdWyXIpCKOcpbF
completed
Usage(completion_tokens=112, prompt_tokens=44, total_tokens=156)
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_CoWSwKPgXJhB5dyTMiTn8pG6'), type='message_creation')




## Retrieving Run Steps

thread_id
(string)

Required
The ID of the thread to which the run and run step belongs.

run_id
(string)

Required
The ID of the run to which the run step belongs.

step_id
(string)

Required
The ID of the run step to retrieve.


### Simple Retrieve
Let's grab a run step using a relatively simple method for a single step

In [26]:
# Retrieve the list of steps
steps_list = client.beta.threads.runs.steps.list(
    thread_id=thread.id, 
    run_id=stream.get_final_run().id
)

# Check if steps_list has an attribute or method to get the first step
first_step = steps_list[0] if isinstance(steps_list, list) else next(iter(steps_list))

# Retrieve the run step
run_step = client.beta.threads.runs.steps.retrieve(
    thread_id=thread.id,
    run_id=stream.get_final_run().id,
    step_id=first_step.id
)

print(run_step)
print("\n")
print(run_step.id)
print(run_step.status)
print(run_step.usage)
print(run_step.step_details.message_creation)
print("\n")

# Get the message associated with the run step
message = client.beta.threads.messages.retrieve(
    thread_id=thread.id, 
    message_id=run_step.step_details.message_creation.message_id
)

# Print the message content
print(message.content[0].text.value)



RunStep(id='step_MM92lE2fUssdWyXIpCKOcpbF', assistant_id='asst_yTqvGZiGrntdsm8J6WDC7xa8', cancelled_at=None, completed_at=1715941562, created_at=1715941559, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_g2Nf7E6dgUQslnsYf7DS8xjQ', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_CoWSwKPgXJhB5dyTMiTn8pG6'), type='message_creation'), thread_id='thread_8lV62hFNMb0jlWQS4P2ovDir', type='message_creation', usage=Usage(completion_tokens=112, prompt_tokens=44, total_tokens=156), expires_at=None)


step_MM92lE2fUssdWyXIpCKOcpbF
completed
Usage(completion_tokens=112, prompt_tokens=44, total_tokens=156)
MessageCreation(message_id='msg_CoWSwKPgXJhB5dyTMiTn8pG6')


A penguin is a flightless bird primarily found in the Southern Hemisphere, particularly in Antarctica. Known for their distinctive black and white plumage, penguins are excellent swimmers, using their flipper-like wings to 

### Retrieving All the Run Steps from Every Run
Now we will look at a technique for getting all the run steps from every run on a thread. 

In [27]:
# Show all the run steps for every run and the message content

# Get a list of runs
runs = client.beta.threads.runs.list(
    thread_id=thread.id,
    order="desc"  # Order runs in descending order
)

# Start our runs loop
for run in runs:
    # List the run steps for the current run
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id
    )
    
    # Start our run steps loop
    for run_step in run_steps:
        print("\n====== Begin Run Step ======\n")
        print(thread.id)
        print("\n")
        print(run.id)
        print("\n")
        print(run_step.id)
        print(run_step.status)
        print(run_step.usage)
        print(run_step.step_details)
        print("\n")

        # Retrieve the message associated with the run step
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=run_step.step_details.message_creation.message_id
        )
        
        # Print the message content
        print(message.content[0].text.value)
        print("\n====== End Run Step ======\n")



====== Begin Run Step ======

thread_8lV62hFNMb0jlWQS4P2ovDir


run_g2Nf7E6dgUQslnsYf7DS8xjQ


step_MM92lE2fUssdWyXIpCKOcpbF
completed
Usage(completion_tokens=112, prompt_tokens=44, total_tokens=156)
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_CoWSwKPgXJhB5dyTMiTn8pG6'), type='message_creation')


A penguin is a flightless bird primarily found in the Southern Hemisphere, particularly in Antarctica. Known for their distinctive black and white plumage, penguins are excellent swimmers, using their flipper-like wings to navigate underwater. They feed mainly on fish, squid, and krill. Penguins are social animals, often forming large colonies for breeding and protection. Their unique adaptations, such as a layer of blubber and tightly packed feathers, provide insulation against the cold. There are around 18 species of penguins, varying in size and habitat preferences.

====== End Run Step ======

